For stage ss it will be "centre_mean" else "w_centre"

In [ ]:
import pandas as pd
import numpy as np
import os
import config

import matplotlib.pyplot as plt
import seaborn as sns

import scikit_posthocs as sp
from scipy.stats import kruskal, combine_pvalues, spearmanr
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_fscore_support, f1_score, cohen_kappa_score, average_precision_score

In [ ]:
def significant_heatmap(result, alpha = 0.05):
    mask = result < alpha
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(result, annot=True, fmt=".3f", cmap='coolwarm_r', cbar = False)
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if not mask[i, j]:
                ax.text(j + 0.5, i + 0.5, '', ha='center', va='center', color='blue')
    plt.show()

def get_pvalues(df):
    pval = df.where(np.triu(np.ones(df.shape), k=1).astype(bool)).stack().values
    _, global_p = combine_pvalues(pval, method='stouffer')
    return pval, global_p

def kruskal_wallis_analysis(df, val_column, cluster_col):
    groups = [group[val_column].values for name, group in df.groupby(cluster_col)]
    stat, p = kruskal(*groups)
    print(f"Kruskal–Wallis H-statistic: {stat:.3f}")
    print(f"p-value: {p:.4f}")

    if stat > 10 and p < 0.05:
        print("Post-hoc Dunn's test results:")
        dunns = sp.posthoc_dunn(df, val_col=val_column, group_col=cluster_col, p_adjust='bonferroni')
        print(f"Combined p-value (Stouffer's method): {get_pvalues(dunns)[1]:.4f}")
        significant_heatmap(dunns.values)
        # display(sp.posthoc_dunn(df, val_col=val_column, group_col=cluster_col, p_adjust='bonferroni'))
        print("Post-hoc Conover's test results:")
        conover = sp.posthoc_conover(df, val_col=val_column, group_col=cluster_col, p_adjust='holm')
        print(f"Combined p-value (Stouffer's method): {get_pvalues(conover)[1]:.4f}")
        # display(sp.posthoc_conover(df, val_col=val_column, group_col=cluster_col, p_adjust='holm'))
        significant_heatmap(conover.values)

def get_metrics(df, label = "cluster_label", score = "mean"):
    results = {}
    labels = df[label].unique().tolist()
    labels.sort()

    res = spearmanr(df[score].tolist(), df[label].tolist())
    results['spearmanr'] = res[0]

    for i in range(len(labels)-1):
        df['binary_label'] = 0
        df.loc[df[label] > labels[i], 'binary_label'] = 1
        fpr, tpr, thresholds = roc_curve(np.array(df['binary_label']),np.array(df[score]))
        auc = metrics.auc(fpr, tpr)
        ap_score = average_precision_score(np.array(df['binary_label']),np.array(df[score]))
       
        results[f'cluster_{labels[i]}'] = {'roc_auc': auc,
                                             'average_precision': ap_score}
        # results[f'cluster_{labels[i]}'] = {'roc_auc': auc,
        #                                    'fpr': fpr,
        #                                    'tpr': tpr}
    return results

In [ ]:
STAGE = 'ss'
#MOD_PREFIX = "mod_2"
MOD_PREFIX = "mod_smallimg"
NEPOCH = 400

#DATAPATH = config.CHENETAL_DATAPATH
DATAPATH = config.OUTPUT_PATH
base_dir = config.RAW_DATA_PATH
outputs = os.path.join(DATAPATH, 'outputs', 'dfs', STAGE)

anomalyscore_metric = "centre_mean"

# Get Data

## Get Aggregated Data

In [ ]:
filepath =  []
for file in os.listdir(outputs):
    if MOD_PREFIX in file and str(NEPOCH) in file:
        filepath.append(os.path.join(outputs, file))

In [ ]:
dfs = []
for path in filepath:
    df = pd.read_csv(path)[['id', anomalyscore_metric]]  # only keep id + target col
    dfs.append(df.rename(columns={anomalyscore_metric: os.path.basename(path)})) 

In [ ]:
combined = dfs[0]
for df in dfs[1:]:
    combined = pd.merge(combined, df, on='id', how="inner")  # 'inner' keeps only common IDs

experiment_cols = [c for c in combined.columns if c != 'id']
combined["mean"] = combined[experiment_cols].mean(axis=1)
combined["std"] = combined[experiment_cols].std(axis=1)

In [ ]:
combined.to_csv(os.path.join(outputs, f"{MOD_PREFIX}_{STAGE}_aggregated_scores.csv"), index = False)

## Get Cluster Data

In [ ]:
folder = "2025-09-25_hdbscan"
# run = "run42"
run = "run74"
folder_date = folder.split('_')[0]

filepath = os.path.join(DATAPATH, folder, "questionnaire", run)
save_path = os.path.join(filepath, "img")
os.makedirs(save_path, exist_ok=True)
df = pd.read_csv(os.path.join(filepath, f'questionnaire_{run}_umap_hdbscan_scaled_wKL.csv'))

In [ ]:
cluster_col = "cluster_label"

# Analysis Anomaly Score

In [ ]:
combined['filepath'] = combined['id']
combined['id'] = combined['id'].apply(lambda x: x.split('/')[-1].replace('.png', ''))

In [ ]:
dfc = combined.merge(df, on='id', how = 'inner')

In [ ]:
print(len(dfc))

In [ ]:
l = list(dfc[cluster_col].unique())
l.sort()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data=dfc, x=cluster_col, y='mean', order=l)

In [ ]:
for cluster in l:
    plt.hist(dfc[dfc[cluster_col]==cluster]['mean'], alpha=0.5, bins=30, label=f'Cluster {cluster}')
    plt.legend()
    plt.title(f'Histogram of Anomaly Scores for Cluster {cluster}')
    plt.xlabel('Anomaly Score')
    plt.ylabel('Frequency')
    plt.show()


## Kruskal Wallis

Null HT: All clusters have the same distribution of anomaly scores.

Alternative HT: At least one cluster has a different distribution.

In [ ]:
groups = [group['mean'].values for name, group in dfc.groupby(cluster_col)]
stat, p = kruskal(*groups)
print(f"Kruskal–Wallis H-statistic: {stat:.3f}")
print(f"p-value: {p:.4f}")

H large -> at least one group differs, if p-value < 0.05

## Post-hoc Pairwise test

Pairwise comparison, to see if only one group significantly differs or multiple groups.

In [ ]:
kruskal_wallis_analysis(dfc, val_column = 'mean', cluster_col = cluster_col)

Shows not significant difference here between 0, 1, and 3

# Comparison to MRI Info

In [ ]:
mri = pd.read_csv(os.path.join(base_dir, '2025-09-25_mrismall.csv'))

In [ ]:
dfc2 = dfc.merge(mri, on='id', how='inner')

## MRI Cartilage Degradation

In [ ]:
kruskal_wallis_analysis(dfc2, 'mri_cart_yn', cluster_col)

## MRI Osteophytes

In [ ]:
kruskal_wallis_analysis(dfc2, 'mri_osteo_yn', cluster_col)

## MRI BML (lesions)

In [ ]:
kruskal_wallis_analysis(dfc2, 'mri_bml_yn', cluster_col)

# ROC_AUC

In [ ]:
results = get_metrics(dfc2, label = cluster_col, score = 'mean')
for i in range(1,len(results.items())):
    print(f"Metrics for cluster_{i-1}:")
    print(results[f'cluster_{i-1}'])

In [ ]:
print(f"Average ROC_AUC across clusters: {np.mean([results[f'cluster_{i}']['roc_auc'] for i in range(len(results)-1)]):.3f}")